In [72]:
import time
import string
import numpy as np
import pandas as pd
import xxhash

In [7]:
df_test = pd.read_csv("Airline Dataset.csv")

In [73]:
import hashlib
import mmh3

keys = df_test['First Name'].astype(str).tolist()
values = df_test['Age'].astype(str).tolist()
key_value_pairs = [(key, value) for key, value in zip(keys, values)]


class HashTable:
    def __init__(self, hash_function, num_buckets):
        self.hash_function = hash_function
        self.table = [None] * num_buckets
        self.num_buckets = num_buckets
    
    def insert(self, key, value):
        hash_value = self.hash_function(key, self.num_buckets)
        index = hash_value % self.num_buckets
        new_entry = (key, value)
        while self.table[index] is not None and self.table[index][0] != key:
            index = (index + 1) % self.num_buckets
        self.table[index] = new_entry

    def retrieve(self, key):
        hash_value = self.hash_function(key, self.num_buckets)
        index = hash_value % self.num_buckets
        while self.table[index] is not None:
            if self.table[index][0] == key:
                return self.table[index][1]
            index = (index + 1) % self.num_buckets
        return None

def murmurhash3(data, _=None): 
    return mmh3.hash(data)

def xxhash_func(data, _=None):
    return xxhash.xxh64(data).intdigest()


# Time measurements
def measure_time(hash_function, num_buckets):
    hash_table = HashTable(hash_function, num_buckets)
    
    # Storing
    start_time = time.time()
    for key, value in key_value_pairs:
        hash_table.insert(key, value)
    store_time = time.time() - start_time
    
    # Retrieving
    start_time = time.time()
    for key, _ in key_value_pairs:
        _ = hash_table.retrieve(key)
    retrieve_time = time.time() - start_time

    return store_time, retrieve_time

In [66]:
prime_buckets = 100003  # A prime number for bucket count
non_prime_buckets = 100002 # Non prime

In [87]:
def simple_sum_hash(key, num_buckets):
    return sum(ord(c) for c in key) % num_buckets

prime_store_simple, prime_retrieve_simple = measure_time(simple_sum_hash, prime_buckets)
non_prime_store_simple, non_prime_retrieve_simple = measure_time(simple_sum_hash, non_prime_buckets)

print("\nSimple Sum Hash:")
print("Prime Buckets - Store:", prime_store_simple, ", Retrieve:", prime_retrieve_simple)
print("Non-prime Buckets - Store:", non_prime_store_simple, ", Retrieve:", non_prime_retrieve_simple)


Simple Sum Hash:
Prime Buckets - Store: 16.262919902801514 , Retrieve: 16.70712900161743
Non-prime Buckets - Store: 16.140865087509155 , Retrieve: 16.80592107772827


In [85]:
murmur_store_time, murmur_retrieve_time = measure_time(murmurhash3, prime_buckets)
murmur_store_time1, murmur_retrieve_time1 = measure_time(murmurhash3, non_prime_buckets)

print("\nxxMurMurHash3:")
print(f"MurmurHash3 (Prime # Buckets) - Store: {murmur_store_time}, Retrieve: {murmur_retrieve_time}")
print(f"MurmurHash3 (Non-Prime # Buckets) - Store: {murmur_store_time1}, Retrieve: {murmur_retrieve_time1}")


xxMurMurHash3:
MurmurHash3 (Prime # Buckets) - Store: 0.05293607711791992, Retrieve: 0.025412797927856445
MurmurHash3 (Non-Prime # Buckets) - Store: 0.020811080932617188, Retrieve: 0.017499923706054688


In [86]:
prime_store_xx, prime_retrieve_xx = measure_time(xxhash_func, prime_buckets)
non_prime_store_xx, non_prime_retrieve_xx = measure_time(xxhash_func, non_prime_buckets)
print("\nxxHash:")
print(f"Prime Buckets - Store: {prime_store_xx}, Retrieve: {prime_retrieve_xx}")
print(f"Non-prime Buckets - Store: {non_prime_store_xx}, Retrieve: {non_prime_retrieve_xx}")


xxHash:
Prime Buckets - Store: 0.061853885650634766, Retrieve: 0.02809309959411621
Non-prime Buckets - Store: 0.026203155517578125, Retrieve: 0.02412891387939453
